<h1>Portfolio Optimisation</h1>
In this project, we will try to implement the conditional portfolio optimisation method for capital allocation. The paper does not explicit state the implementation of this, so we will try to prototype it with python using machine learning methods. The model will be given a list of tickers which are highly probable to be profitable, and then we use conditional portfolio optimisation to optimise the capital allocation according to current market sentiment.

Part 1, we will first try to find the tickers with highest probability of profit by implementing the method outlined in the paper in the Journal of Forecasting.

Part 2, we will use previously collected tickers to form a optimised allocation of capital. In order for this to work, essentially we plug in macroeconomic data to represent the general market sentiment and also the allocation of tickers which will be generated as a grid, then we get a predicted output which will be the thing we are trying to maximise, in this example we will try to maximise 60 day sharpe ratio, annualised return (365D) <-> alpha, also the weighted sum of sharpe ratio and return (want to account for both), and we just rank the output and use the combination with the highest objective value. However, we note that the complexity of the problem scales exponentially which means that we will need to be selective in terms of the grid search when searching for optimal allocations.

<h1>Part 1: Stock selection with machine learning</h1>
As outlined in the paper, we will select stock tickers using several different machine learning models and also an ensemble model (which had shown to produced best returns). For simplicity sake, we will only be using stocks listed on NASDAQ (5044 tickers)

In [4]:
import requests
from IPython.display import clear_output
import json
def get_fundamental_data(ticker_list):
    progress = 1
    for i in range(len(ticker_list)):
        # Fetch Data
        url = f"https://eodhd.com/api/fundamentals/{ticker_list[i]}?api_token=667822cc36e777.79338265&fmt=json"
        try:
            response = requests.get(url).json()
        except:
            continue
        append_to_file(response)

        # Show progression
        progress = (i/len(ticker_list)*100)
        num_stars = int(progress//10 + 1)
        print(num_stars * "*" + (10 - num_stars) * "-", f"{progress:.2f}%", flush=True)
        clear_output(wait=True)
    


def append_to_file(new_data, filename="fundamental_data.json"):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["result"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

In [5]:
import pandas as pd
import requests
import numpy as np
import json

# Download stock tickers listed on NASDAQ (use if NASDAQ have new listings)
"""
EXCHANGE_CODE="NASDAQ"
url = f"https://eodhd.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token=667822cc36e777.79338265&fmt=json"
response = requests.get(url)
ticker_data = pd.DataFrame(response.json())
ticker_data.to_csv("tickers.csv", index=False)
ticker_data
"""

# Load stock tickers
ticker_data = pd.read_csv("tickers.csv")

# Load into list
ticker_list = list(ticker_data["Code"] + ".US") # Easier to plug into EOD api call

data = get_fundamental_data(ticker_list)


KeyboardInterrupt: 

In [10]:
json_object = json.dumps(data, indent=4)
with open("fundamental_data.json", "w") as file:
    file.write(json_object)

<h1>Part 2: Conditional Portfolio Optimisation</h1>

In [51]:
# We first want to define a few objective functions we are trying to maximise
def alpha(rp: float, rm: float, rf_rate: float, beta: float) -> float:
    """
    rp: the return of the portfolio
    rm: the return of the market
    rf_rate: the risk free rate
    beta: assets beta
    Calculate alpha of given asset return.
    """
    return (rp - rf_rate - beta * (rm - rf_rate))

def beta(asset_price_series:list[float], market_price_series:list[float]) -> float:
    if len(asset_price_series) > len(market_price_series):
        asset_price_series = asset_price_series[len(asset_price_series) - len(market_price_series):]
    elif len(market_price_series) > len(asset_price_series):
        market_price_series = market_price_series[len(market_price_series) - len(asset_price_series):]
    covariance: float = np.sum((asset_price_series - np.mean(asset_price_series)) * (market_price_series - np.mean(market_price_series))) / len(asset_price_series)
    variance: float = np.var(market_price_series)
    return covariance / variance

# Demonstration
np.random.seed(123)
series_1 = np.random.randn(10)

series_2 = np.random.randn(11)
b = beta(series_1, series_2)
rp = (series_1[-1] - series_1[0]) / series_1[0]
rm = (series_2[-1] - series_2[0]) / series_2[0]
rf_rate = 4.2/100
print(series_1)
print(series_2)
alpha(rp, rm, rf_rate, b)

[-1.0856306   0.99734545  0.2829785  -1.50629471 -0.57860025  1.65143654
 -2.42667924 -0.42891263  1.26593626 -0.8667404 ]
[-0.67888615 -0.09470897  1.49138963 -0.638902   -0.44398196 -0.43435128
  2.20593008  2.18678609  1.0040539   0.3861864   0.73736858]


0.13102032113541398